# Week 2 Assignment: CIFAR-10 Autoencoder

## Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

2024-07-29 14:14:37.119736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 14:14:37.119840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 14:14:37.261002: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load and prepare the dataset

In [2]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

In [8]:
# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024


### START CODE HERE (Replace instances of `None` with your code) ###

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10', as_supervised=True, split='train')

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10', as_supervised=True, split="test")

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

### END CODE HERE ###

## Build the Model

In [9]:
# suggested layers to use. feel free to add or remove as you see fit.
from keras.layers import Conv2D, UpSampling2D, MaxPooling2D

def get_autoencoder_model(input_shape):
  model = Sequential([
                      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape), 
                      MaxPooling2D(pool_size=(2, 2)), 
                      Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'), 
                      MaxPooling2D(pool_size=(2, 2)), 
                      Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'), 
                      Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'), 
                      UpSampling2D(size=(2, 2)), 
                      Conv2D(filters=64, kernel_size=(3, 3), activation ='relu', padding='same'), 
                      UpSampling2D(size=(2, 2)), 
                      Conv2D(filters=3, kernel_size=(3, 3), activation ='sigmoid', padding='same'), 
  ])
  return model

model = get_autoencoder_model((32, 32, 3))
### END CODE HERE ###

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 8, 8, 128)      │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 3)      │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 741,379 (2.83 MB)

 Trainable params: 741,379 (2.83 MB)

 Non-trainable params: 0 (0.00 B)

## Configure training parameters

In [10]:
# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training

In [11]:
# parameters (feel free to change this)
train_steps = len(train_dataset) // BATCH_SIZE 
val_steps = len(test_dataset) // BATCH_SIZE

### START CODE HERE ###
model.fit(train_dataset, steps_per_epoch=train_steps, validation_data=test_dataset, validation_steps=val_steps, epochs=50)
### END CODE HERE ###

Epoch 1/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 15s 8s/step - accuracy: 0.3104 - loss: 0.0682

I0000 00:00:1722262753.814038     110 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.3662 - loss: 0.0671 - val_accuracy: 0.2868 - val_loss: 0.0607
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 258ms/step - accuracy: 0.3935 - loss: 0.0572 - val_accuracy: 0.4121 - val_loss: 0.0510
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - accuracy: 0.4256 - loss: 0.0554 - val_accuracy: 0.3337 - val_loss: 0.0500
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step - accuracy: 0.4137 - loss: 0.0486 - val_accuracy: 0.4910 - val_loss: 0.0413
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step - accuracy: 0.4873 - loss: 0.0397 - val_accuracy: 0.5406 - val_loss: 0.0347
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 245ms/step - accuracy: 0.5070 - loss: 0.0336 - val_accuracy: 0.4510 - val_loss: 0.0330
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.4270 - loss: 0.0319 - val_accuracy: 0.4788 - val_loss: 0.0299
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step - accuracy: 0.5012 - loss: 0.0303 - val_accuracy: 0.5349 - val_loss: 0.0277
Epoch

## Model evaluation

In [12]:
result = model.evaluate(test_dataset, steps=10)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7370 - loss: 0.0071


## Save the Model

In [13]:
# Save the model you just trained
model.save("temp_model.h5")